In [1]:
import keras as K
import tensorflow as tf

In [13]:
classes = ['dry_asphalt_severe',
           'dry_asphalt_slight',
           'dry_asphalt_smooth',
           'dry_concrete_severe',
           'dry_concrete_slight',
           'dry_concrete_smooth',
           'water_asphalt_severe',
           'water_asphalt_slight',
           'water_asphalt_smooth',
           'water_concrete_severe',
           'water_concrete_slight',
           'water_concrete_smooth',
           'wet_asphalt_severe',
           'wet_asphalt_slight',
           'wet_asphalt_smooth',
           'wet_concrete_severe',
           'wet_concrete_slight',
           'wet_concrete_smooth']

batch_size = 32

num_classes =len(classes)

dataset_path = './datasets'

In [4]:
from keras.preprocessing.image import ImageDataGenerator

def generators(shape, preprocessing): 
    '''Create the training and validation datasets for 
    a given image shape.
    '''
    imgdatagen = ImageDataGenerator(
        preprocessing_function = preprocessing,
        horizontal_flip = True, 
        validation_split = 0.1,
    )

    height, width = shape

    train_dataset = imgdatagen.flow_from_directory(
        dataset_path,
        target_size = (height, width), 
        classes = classes,
        batch_size = batch_size,
        subset = 'training', 
    )

    val_dataset = imgdatagen.flow_from_directory(
        dataset_path,
        target_size = (height, width), 
        classes = classes,
        batch_size = batch_size,
        subset = 'validation'
    )
    return train_dataset, val_dataset

In [12]:
len(classes)

18

In [9]:
train_dataset, val_dataset = generators(
    (224, 224), preprocessing=K.applications.resnet50.preprocess_input)

Found 16200 images belonging to 18 classes.
Found 1800 images belonging to 18 classes.


In [14]:
inputs = K.Input(shape=(224, 224, 3))

"""Loading the ResNet50 model with pre-trained ImageNet weights
"""
resnet = K.applications.ResNet50(weights='imagenet',
                                 include_top=False, input_tensor=inputs)

for layer in resnet.layers[:170]:
    layer.trainable = False

model = K.models.Sequential()
model.add(K.layers.Lambda(lambda x: tf.image.resize(x, (224, 224))))
model.add(resnet)
model.add(K.layers.GlobalAveragePooling2D())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(256, activation='relu'))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(128, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(64, activation='relu'))
model.add(K.layers.Dropout(0.3))
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dense(num_classes, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=K.optimizers.RMSprop(learning_rate=2e-5),
              metrics=['accuracy'])
checkpointer = K.callbacks.ModelCheckpoint(filepath='rts-checkpoint.keras',
                                           monitor="val_accuracy",
                                           verbose=1, save_best_only=True)

In [ ]:
history = model.fit(train_dataset, validation_data=val_dataset,
                    epochs=10, callbacks=[checkpointer])


In [ ]:
model.summary()

model.save("rts.keras")